In [3]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import colors

import numpy as np
import xarray as xr

### Pattern Scaling

https://link.springer.com/article/10.1007/s10584-013-1032-9

https://web.stanford.edu/group/emf-research/docs/sm/2016/ClaudiaT_PatternScaling.pdf

Search assist says **NOTE THIS IS AI**t- 


Pattern scaling is a method used in climate science to estimate how changes in global temperature affect regional climate variables, like precipitation and temperature. It involves using existing climate model outputs to create simplified projections for future climate scenarios, making it easier and less costly to analyze potential climate impacts.

says only need

the actual GCM field (tas_truth)

the global mean temperature (tas_global)

In [4]:
# define your own data_path
data_path = "./dataset/"

#Get test data (SSP 245)
test_Y = xr.open_dataset(data_path + 'outputs_ssp245.nc').compute()
test_X = xr.open_dataset(data_path + 'inputs_ssp245.nc').compute()

# Get one combined historical + ssp585 timeseries for now
X = xr.open_mfdataset([data_path + 'inputs_historical.nc', data_path + 'inputs_ssp585.nc']).compute()
# Take the 2nd ensemble member for the historical (the first one has some missing DTR values for some reason...) and the 1st (only) one for SSP585
Y = xr.concat([xr.open_dataset(data_path + 'outputs_historical.nc').sel(member=2), xr.open_dataset(data_path + 'outputs_ssp585.nc').sel(member=1)], dim='time').compute()

# Convert the precip values to mm/day
Y["pr"] *= 86400
Y["pr90"] *= 86400

In [8]:
def compute_global_mean(field):
    # area weights (cos(lat))
    weights = np.cos(np.deg2rad(field.lat))
    weights /= weights.mean()

    return (field * weights).mean(dim=("lat", "lon"))

def compute_pattern_scaling(truth, global_ts, baseline=(1850,1900)):
    # anomalies
    da = truth - truth.sel(time=slice(*baseline)).mean("time")
    gm = global_ts - global_ts.sel(time=slice(*baseline)).mean("time")

    # regression slope at each grid cell = pattern
    S = (da * gm).mean("time") / (gm**2).mean("time")
    return S

def rmse(true, pred):
    return float(np.sqrt(((true - pred) ** 2).mean()))

- called single regresser pattern scaling
- Y(x,y,t)≈S(x,y)⋅G(t)
    - Y is the local climate variable (tas).
    - G(t) is the global mean temperature.
    - S(x,y) is the regression slope at each grid cell.

In [19]:
tas_global = compute_global_mean(Y["tas"])
tas_global_TEST = compute_global_mean(test_Y["tas"])

S_tas = compute_pattern_scaling(Y["tas"], tas_global)

tas_scaled_TEST = S_tas * tas_global_TEST

rmse_tas = rmse(test_Y["tas"], tas_scaled_TEST)
print("RMSE tas:", rmse_tas)

RMSE tas: 0.7171458521812378


this is the simplest model that I could make in 2 hours after learning what pattern scaling is. not great

A model involving linear regression will be looked at later
- also involving data from SSP126, 370 and 585.

the provided notebook on repo is a good baseline.

